In [2]:
import os, sys, shutil
from tqdm import tqdm
import numpy as np
import itertools
import pandas as pd
# Set the display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib as plt
from PIL import Image
from matplotlib.lines import Line2D
import math
import random
from datetime import datetime
from scipy.stats import binomtest
from collections import defaultdict
experiment_version = 1
response_version = 1
stimuli_path = f"stimuli_v{experiment_version}/"
response_path = f"responses_v{experiment_version}/"
dataframe_path = f"dataframes_v{experiment_version}/"

# CREATE EXPERIMENT DATAFRAME AND TRIAL FILES FOR MEADOWS

In [ ]:
os.makedirs(stimuli_path, exist_ok=True)
os.makedirs(response_path, exist_ok=True)
os.makedirs(dataframe_path, exist_ok=True)
#Experiment column key:
# 1: Experiment 1, 2AFC identification for all reconstruction methods
# 2: Experiment 2, 2AFC vision vs imagery for all reconstruction methods
# 3: Experiment 3, Similarity Score vision vs imagery
# 4: Experiment 4, Similarity Score method comparison on conceptual stimuli
df_exp = pd.DataFrame(columns=["experiment", "stim1", "stim2", "stim3", "sample", "subject", "target_on_left", "method", "catch_trial", "rep", "mode", "stimtype"])
i=0
random_count = 0
stimuli_root = "../output"
for subj in [1,2,5,7]: #1,2,5,7
    #Experiment 1, mental imagery two way identification
    for mode in ["vision", "imagery"]:
        if mode == "vision":
            num_samples = 12
        else:
            num_samples = 18
        for sample in tqdm(range(num_samples)):
            gt_sample = f"{sample}_ground_truth"
            if sample < 6:
                stimtype = "simple"
            elif sample < 12:
                stimtype = "complex"
            else:
                stimtype = "concept"
            for method in ["mirage", "iCNN", "mindeye", "braindiffuser", "mindeye2", "takagi"]: 
                for rep in range(10):
                    
                    # Get random sample to compare against
                    if stimtype == "simple":
                        random_sample = random.choice([x for x in range(6) if x != sample])
                    elif stimtype == "complex":
                        random_sample = random.choice([x for x in range(6, 12) if x != sample])
                    else:
                        random_sample = random.choice([x for x in range(12, 18) if x != sample])
                    random_rep = random.choice([x for x in range(10)])
                    
                    sample_recon = f"{sample}_{rep}_{mode}_subject{subj}_{method}"
                    random_recon = f"{random_sample}_{random_rep}_{mode}_subject{subj}_{method}"
                    
                    # Load the stimulus images and save as pngs to stimuli folder
                    gt_sample_path = f"{stimuli_root}vision/{method}/subject{subj}/{sample}/ground_truth.png"
                    sample_recon_path = f"{stimuli_root}{mode}/{method}/subject{subj}/{sample}/{rep}.png"
                    random_recon_path = f"{stimuli_root}{mode}/{method}/subject{subj}/{random_sample}/{random_rep}.png"
                    
                    # Copy the stimulus images to the stimuli folder
                    shutil.copy(gt_sample_path, f"stimuli_v{experiment_version}/{gt_sample}.png")
                    shutil.copy(sample_recon_path, f"stimuli_v{experiment_version}/{sample_recon}.png")
                    shutil.copy(random_recon_path, f"stimuli_v{experiment_version}/{random_recon}.png")
        
                    # Configure stimuli names and order in experiment dataframe
                    order = random.randrange(2)
                    sample_names = [sample_recon, random_recon]
                    left_sample = sample_names.pop(order)
                    right_sample = sample_names.pop()
                    
                    df_exp.loc[i] = {"experiment" : 1, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, "target_on_left" : order == 0, "method" : method, "catch_trial" : None, "rep" : rep, "mode" : mode, "stimtype" : stimtype, "trial_rep" : 0}
                    i+=1
    #Experiment 2, 2AFC vision vs imagery for all reconstruction methods
    for sample in tqdm(range(12)):
        gt_sample = f"{sample}_ground_truth"
        for method in ["mirage", "iCNN", "mindeye", "braindiffuser", "mindeye2", "takagi"]: 
            for rep in range(10):
                vision_recon = f"{sample}_{rep}_vision_subject{subj}_{method}"
                imagery_recon = f"{sample}_{rep}_imagery_subject{subj}_{method}"
                
                # Load the stimulus images and save as pngs to stimuli folder
                gt_sample_path = f"{stimuli_root}vision/{method}/subject{subj}/{sample}/ground_truth.png"
                vision_recon_path = f"{stimuli_root}vision/{method}/subject{subj}/{sample}/{rep}.png"
                imagery_recon_path = f"{stimuli_root}imagery/{method}/subject{subj}/{sample}/{rep}.png"
    
                # Configure stimuli names and order in experiment dataframe
                order = random.randrange(2)
                sample_names = [vision_recon, imagery_recon]
                left_sample = sample_names.pop(order)
                right_sample = sample_names.pop()
                if sample < 6:
                    stimtype = "simple"
                elif sample < 12:
                    stimtype = "complex"
                else:
                    stimtype = "concept"
                # "Target" is the vision reconstruction
                df_exp.loc[i] = {"experiment" : 2, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, "target_on_left" : order == 0, "method" : method, "catch_trial" : None, "rep" : rep, "mode" : mode, "stimtype" : stimtype, "trial_rep" : 0}
                i+=1
    # Experiment 3: Vision vs Imagery similarity comparison w/ the Drag-Rate task
    for sample in tqdm(range(12)):
        gt_sample = f"{sample}_ground_truth"
        for method in ["mirage", "iCNN", "mindeye", "braindiffuser", "mindeye2", "takagi"]: 
            for rep in range(10):
                vision_recon = f"{sample}_{rep}_vision_subject{subj}_{method}"
                imagery_recon = f"{sample}_{rep}_imagery_subject{subj}_{method}"
                
                # Load the stimulus images and save as pngs to stimuli folder
                gt_sample_path = f"{stimuli_root}vision/{method}/subject{subj}/{sample}/ground_truth.png"
                vision_recon_path = f"{stimuli_root}vision/{method}/subject{subj}/{sample}/{rep}.png"
                imagery_recon_path = f"{stimuli_root}imagery/{method}/subject{subj}/{sample}/{rep}.png"
    
                # Configure stimuli names and order in experiment dataframe
                order = random.randrange(2)
                sample_names = [vision_recon, imagery_recon]
                left_sample = sample_names.pop(order)
                right_sample = sample_names.pop()
                if sample < 6:
                    stimtype = "simple"
                else:
                    stimtype = "complex"
                df_exp.loc[i] = {"experiment" : 3, "stim1" : gt_sample, "stim2" : left_sample, "stim3" : right_sample, "sample" : sample, "subject" : subj, 
                                "target_on_left" : order == 0, "method" : method, "catch_trial" : None, "rep" : rep, "mode" : "both", "stimtype" : stimtype}
                i+=1
    # Experiment 4: Method comparison for conceptual stimuli on similarity score task
    for sample in tqdm(range(12, 18)):  # samples 12 to 17 (conceptual stimuli)
        gt_sample = f"{sample}_ground_truth"
        stimtype = "concept"
        for rep in range(10):
            # Generate all possible method pairs (15 pairs)
            method_pairs = list(itertools.combinations(["mirage", "iCNN", "mindeye", "braindiffuser", "mindeye2", "takagi"], 2))
            # Randomly select 9 pairs per sample per subject per repetition
            selected_pairs = random.sample(method_pairs, 9)
            for method1, method2 in selected_pairs:
                # Construct filenames for reconstructions
                recon1 = f"{sample}_{rep}_imagery_subject{subj}_{method1}"
                recon2 = f"{sample}_{rep}_imagery_subject{subj}_{method2}"
                
                # Paths to the images
                gt_sample_path = f"{stimuli_root}imagery/{method1}/subject{subj}/{sample}/ground_truth.png"
                recon1_path = f"{stimuli_root}imagery/{method1}/subject{subj}/{sample}/{rep}.png"
                recon2_path = f"{stimuli_root}imagery/{method2}/subject{subj}/{sample}/{rep}.png"
                
                # Configure stimuli names, methods are already randomized
                sample_names = [recon1, recon2]
                left_sample = recon1
                right_sample = recon2
                
                # Record the trial in the dataframe
                df_exp.loc[i] = {"experiment": 4, "stim1": gt_sample, "stim2": left_sample, "stim3": right_sample, "sample": sample, "subject": subj, "target_on_left": None, "method": f"{method1}_vs_{method2}", "catch_trial": None, "rep": rep, "mode": "imagery", "stimtype": stimtype, "trial_rep": 0}
                i += 1
df_exp = df_exp.sample(frac=1)
print(len(df_exp))


In [ ]:
# Check if all images are present in final stimuli folder
count_not_found = 0
stim_path = f"stimuli_v{experiment_version}/"
for index, row in df_exp.iterrows():
    if not (os.path.exists(f"{stim_path}{row['stim1']}.png")):
        print(f"{row['stim1']}.png")
        count_not_found += 1
    if not (os.path.exists(f"{stim_path}{row['stim2']}.png")):
        print(f"{row['stim2']}.png")
        count_not_found += 1
    if not (os.path.exists(f"{stim_path}{row['stim3']}.png")):
        print(f"{row['stim3']}.png")
        count_not_found += 1
print(count_not_found)

### Add pID columns to assign trials to different participants

In [32]:
# Shuffle Experiments 1, 2, and 3 individually
df_exp12 = df_exp[df_exp['experiment'].isin([1,2])].sample(frac=1, random_state=42)
df_exp34 = df_exp[df_exp['experiment'].isin([3, 4])].sample(frac=1, random_state=42)

# Calculate the number of participants needed
exp12_trials_per_participant = 20
exp34_trials_per_participant = 10
num_participants = max(len(df_exp12) // exp12_trials_per_participant, len(df_exp34) // exp34_trials_per_participant)

# Assign pID for Experiment 1
df_exp12 = df_exp12.sample(frac=1, random_state=42)  # Re-shuffle to mix experiments 1 and 2
df_exp12['pID'] = [i % num_participants for i in range(len(df_exp12))]

# Shuffle df_exp34 again before assigning pID to ensure randomness in distribution
df_exp34 = df_exp34.sample(frac=1, random_state=42)  # Re-shuffle to mix experiments 3 and 4
df_exp34['pID'] = [i % num_participants for i in range(len(df_exp34))]

# Combine and shuffle the dataframe to mix up the trials across all experiments
df_exp_pid = pd.concat([df_exp12, df_exp34]).sample(frac=1, random_state=42)

# Sort by pID to ensure the dataframe is ordered by participant, facilitating even distribution
df_exp_pid.sort_values(by='pID', inplace=True)

# Ensure pID is the first column
cols = list(df_exp_pid.columns)
cols.insert(0, cols.pop(cols.index('pID')))
df_exp_pid = df_exp_pid[cols]

In [ ]:
#Add catch trials within each pID section
df_exp_catch = df_exp_pid.copy()
for pID in np.unique(df_exp_catch['pID']):
    for experiment_list, num_catch in zip([[1, 2], [3, 4]], [4, 2]): 
        df_pid = df_exp_catch[(df_exp_catch['experiment'].isin(experiment_list)) & (df_exp_catch['pID'] == pID)]
        
        # Ground truth catch trials
        try:
            gt_catch_trials = df_pid.sample(n=num_catch)
        except:
            print(pID, df_pid)
        gt_catch_trials['catch_trial'] = "ground_truth"
        for index, row in gt_catch_trials.iterrows():
            
            order = random.randrange(2)
            ground_truth = row['stim1']
            stims = [ground_truth, row['stim2']]
            
            gt_catch_trials.at[index, 'stim2'] = stims.pop(order)
            gt_catch_trials.at[index, 'stim3'] = stims.pop()
            # Target on left here means the ground truth repeat is on the left
            gt_catch_trials.at[index, 'target_on_left'] = (order == 0)
        df_exp_catch = pd.concat([df_exp_catch, gt_catch_trials])
# shuffle catch trials into the sessions
df_exp_catch = df_exp_catch.sample(frac=1).sort_values(by='pID', kind='mergesort')
print(len(df_exp_catch))
print(len(df_exp_catch[(df_exp_catch['pID'] == 0)]))
print(len(np.unique(df_exp_catch['pID'])))
print(len(df_exp_catch[df_exp_catch['experiment'].isin([1, 2])]))
print(len(df_exp_catch[df_exp_catch['experiment'].isin([3, 4])]))

### Save experiment dataframe and tsv files

In [ ]:

df_exp_catch.to_csv(f'dataframes_v{experiment_version}/experiment_v{experiment_version}.csv', index=False)
df_catch_exp12 = df_exp_catch[df_exp_catch['experiment'].isin([1, 2])]
df_catch_exp34 = df_exp_catch[df_exp_catch['experiment'].isin([3, 4])]

df_exp_tsv12 = df_catch_exp12[['pID', 'stim1', 'stim2', 'stim3']].copy()
df_exp_tsv34 = df_catch_exp34[['pID', 'stim1', 'stim2', 'stim3']].copy()
df_exp_tsv12.to_csv(f"dataframes_v{experiment_version}/meadow_trials_v{experiment_version}_exp12.tsv", sep="\t", index=False, header=False) 
df_exp_tsv34.to_csv(f"dataframes_v{experiment_version}/meadow_trials_v{experiment_version}_exp34.tsv", sep="\t", index=False, header=False) 

# THE FOLLOWING CELLS ARE FOR PROCESSING RESPONSES

In [154]:
df_experiment = pd.read_csv(dataframe_path + f"experiment_v{experiment_version}.csv")

df_responses = pd.read_csv(f"{response_path}annotations_v{response_version}.csv")

### Parse the responses and associate them with the trials in the experiment dataframe, to have all information available in one dataframe

In [ ]:

# Initialize a list to hold row dictionaries before creating the final dataframe
rows_list = []
df_2afc = df_responses[df_responses["task"] == "Match-To-Sample"]

df_similarity = df_responses[df_responses["task"] == "Drag-And-Rate"]
# Parse trials in 2AFC experiment
for index, row in tqdm(df_2afc.iterrows()):
    if row['label'] == row['stim2_id']:
        picked_left = True
    elif row['label'] == row['stim3_id']:
        picked_left = False
    else:
        print("Error")
        break
    start_timestamp = row['time_trial_start']
    end_timestamp = row['time_trial_response']
    start = datetime.fromisoformat(start_timestamp.replace("Z", "+00:00"))
    end = datetime.fromisoformat(end_timestamp.replace("Z", "+00:00"))
    # Calculate the difference in seconds
    response_time = (end - start).total_seconds()
    
    experiment_row = df_experiment[(df_experiment['experiment'].isin([1,2])) & (df_experiment['stim1'] == row['stim1_name']) & (df_experiment['stim2'] == row['stim2_name']) & (df_experiment['stim3'] == row['stim3_name'])].iloc[0]
    row_data = {
        **experiment_row.to_dict(),
        "picked_left": picked_left,
        "picked_target": picked_left == experiment_row['target_on_left'],
        "participant": row['participation'],
        "response_time": response_time,
    }
    rows_list.append(row_data)
# Parse trials in similarity range experiment by iterating through df_responses two rows at a time, since each stimuli has its own row
for index in tqdm(range(0, len(df_similarity), 2)):
    row1 = df_similarity.iloc[index]
    row2 = df_similarity.iloc[index + 1]
    # Ensure the two rows belong to the same trial
    assert row1["trial"] == row2["trial"], "Rows do not belong to the same trial"
    
    # Attempt to find a matching experiment row
    match1 = df_experiment[(df_experiment['experiment'].isin([3,4])) & (df_experiment['stim2'] == row1['stim1_name']) & (df_experiment['stim3'] == row2['stim1_name'])]
    match2 = df_experiment[(df_experiment['experiment'].isin([3,4])) & (df_experiment['stim2'] == row2['stim1_name']) & (df_experiment['stim3'] == row1['stim1_name'])]
    
    # Determine which match is correct
    if not match1.empty:
        correct_match = match1
        stim2_row, stim3_row = row1, row2
    elif not match2.empty:
        correct_match = match2
        stim2_row, stim3_row = row2, row1
    else:
        continue  # Skip if no correct match is found
    
    # Extract the correct match's data
    experiment_row = correct_match.iloc[0]
    
    # Calculate response times
    start_timestamp = row1['time_trial_start']
    end_timestamp1 = row1['time_trial_response']
    end_timestamp2 = row2['time_trial_response']
    start = datetime.fromisoformat(start_timestamp.replace("Z", "+00:00"))
    end1 = datetime.fromisoformat(end_timestamp1.replace("Z", "+00:00"))
    end2 = datetime.fromisoformat(end_timestamp2.replace("Z", "+00:00"))
    end = max(end1, end2)
    response_time = (end - start).total_seconds()
    
    # Determine if the left stimulus was picked based on the 'y' value
    picked_left = stim2_row['y'] > stim3_row['y']
    if experiment_row['experiment'] == 4:
        methods = experiment_row['method'].split("_vs_")
        stim2_method = methods[0]
        stim3_method = methods[1]
        picked_method = stim2_method if picked_left else stim3_method
    else:
        stim2_method = experiment_row['method']
        stim3_method = experiment_row['method']
        picked_method = None
    
    # Determine target and distractor based on target_on_left flag
    if experiment_row['target_on_left']:
        target_similarity, target_confidence = stim2_row['y'], stim2_row['x']
        distractor_similarity, distractor_confidence = stim3_row['y'], stim3_row['x']
    else:
        target_similarity, target_confidence = stim3_row['y'], stim3_row['x']
        distractor_similarity, distractor_confidence = stim2_row['y'], stim2_row['x']
    
    # Determine if the target was picked
    picked_target = target_similarity > distractor_similarity
    
    # Compile the row data
    row_data = {
        **experiment_row.to_dict(),
        "picked_left": picked_left,
        "participant": row1['participation'],
        "response_time": response_time,
        "stim2_similarity": stim2_row['y'],
        "stim2_confidence": stim2_row['x'],
        "stim3_similarity": stim3_row['y'],
        "stim3_confidence": stim3_row['x'],
        "target_similarity": target_similarity,
        "target_confidence": target_confidence,
        "distractor_similarity": distractor_similarity,
        "distractor_confidence": distractor_confidence,
        "picked_target": picked_target,
        "stim2_method": stim2_method,
        "stim3_method": stim3_method,
        "picked_method": picked_method
    }
    rows_list.append(row_data)

# Create the final dataframe from the list of row dictionaries
df_trial_combined = pd.DataFrame(rows_list)

# Dropping the extra index columns added from the experiment_row.to_dict() conversion
df_trial = df_trial_combined.drop(columns=[col for col in df_trial_combined.columns if 'Unnamed' in col])

### Find trials that didn't get completed in the previous round of the experiment, and save new trial files for another round

In [ ]:
# Initialize a dictionary to count processed trials per pID
processed_trials_count = defaultdict(int)

# Your existing processing loop goes here...
# Update processed_trials_count for each row processed
for index, row_data in df_trial.iterrows():
    processed_trials_count[row_data['pID']] += 1

print(processed_trials_count)

# Count expected trials per pID from df_experiment
expected_trials_count = df_experiment['pID'].value_counts().to_dict()

# Find pIDs with missing trials by comparing processed and expected counts
missing_trials_pids = {pID: expected_trials_count[pID] - processed_trials_count[pID] 
                       for pID in expected_trials_count 
                       if pID not in processed_trials_count or processed_trials_count[pID] < expected_trials_count[pID]-2}

# Print or handle pIDs with missing trials as needed
print(f"missing {len(missing_trials_pids.values())} pIDs")
for pID, missing_count in missing_trials_pids.items():
    print(f"pID: {pID} has {missing_count} missing trials.")
    
if len(missing_trials_pids.values()) > 0:
    df_exp_missing = df_experiment[df_experiment['pID'].isin(missing_trials_pids.keys())]

    df_catch_exp12_missing = df_exp_missing[df_exp_missing['experiment'].isin([1, 2])]
    df_catch_exp34_missing = df_exp_missing[df_exp_missing['experiment'].isin([3, 4])]


    df_exp_tsv12_missing = df_catch_exp12_missing[['pID', 'stim1', 'stim2', 'stim3']].copy()
    df_exp_tsv34_missing = df_catch_exp34_missing[['pID', 'stim1', 'stim2', 'stim3']].copy()
    df_exp_tsv12_missing.to_csv(f"dataframes_v{experiment_version}/meadow_trials_v{experiment_version}_exp12_missing.tsv", sep="\t", index=False, header=False) 
    df_exp_tsv34_missing.to_csv(f"dataframes_v{experiment_version}/meadow_trials_v{experiment_version}_exp34_missing.tsv", sep="\t", index=False, header=False) 

### Parse and remove participants that failed at least 2 catch trials

In [ ]:
# number of participants
print("Total participants:", len(df_trial["participant"].unique()))
gt_failures = df_trial[(df_trial['catch_trial'] == 'ground_truth') & (df_trial['picked_target'] == False)].groupby('participant').size()
# print(df_trial[(df_trial['catch_trial'] == 'ground_truth')])
# Identify participants who failed more than 1 ground truth catch trial
participants_to_remove_rule1 = gt_failures[gt_failures > 1].index.tolist()
print("Participants to remove 1:", participants_to_remove_rule1)

participants_to_remove = set(participants_to_remove_rule1)#.union(set(participants_to_remove_rule2))
filtered_df = df_trial[~df_trial['participant'].isin(participants_to_remove)]
print("Clean participants:", len(filtered_df["participant"].unique()))
print(len(df_trial), len(filtered_df))
print(participants_to_remove)
# Filter out catch trials
filtered_df = filtered_df[(filtered_df['catch_trial'].isnull())]
filtered_df.to_csv(f'{dataframe_path}filtered_responses_v{response_version}.csv', index=False)

# THE FOLLOWING CELLS ARE FOR ANALYZING RESPONSES

In [ ]:
# Load filtered responses
filtered_df = pd.read_csv(f'{dataframe_path}filtered_responses_v{response_version}.csv')
# df_experiment = pd.read_csv(dataframe_path + f"experiment_v{experiment_version}.csv")

# df_responses = pd.read_csv(f"{response_path}annotations_v{response_version}.csv")
experiment = 1
df_trial_exp = filtered_df[(filtered_df['catch_trial'].isnull()) & (filtered_df['experiment'] == experiment)]

# Iterate over each method
for method in df_trial_exp['method'].unique():
    print(f"Method: {method}")
    print("--------------------")
    for mode in ["vision", "imagery"]:
        
        print(f"Mode: {mode}")
        print("--------------------")
        df_trial_exp1 = df_trial_exp[(df_trial_exp['method'] == method) & (df_trial_exp['experiment'] == experiment) & (df_trial_exp['mode'] == mode)]
        # Perform a binomial test
        # The null hypothesis is that the probability of success is 0.5 (chance level)
        if len(df_trial_exp1) == 0:
            print("No trials for this combination")
            continue
        p_value = binomtest(df_trial_exp1['picked_target'].sum(), n=len(df_trial_exp1['picked_target']), p=0.5, alternative='two-sided').pvalue

        print("Number of experiment trials:", len(df_trial_exp1))
        print("Success rate: ", len(df_trial_exp1[df_trial_exp1["picked_target"]]) / len(df_trial_exp1))
        print(f'P-value: {p_value}')
        
        for stimtype in ["simple", "complex", "concept"]:
            print(f"Stimtype: {stimtype}")
            
            print("--------------------")
            # Filter the data for the current method, mode, and experiment
            df_trial_exp1 = df_trial_exp[(df_trial_exp['method'] == method) & (df_trial_exp['experiment'] == experiment) & (df_trial_exp['mode'] == mode) & (df_trial_exp['stimtype'] == stimtype)]
            # Perform a binomial test
            # The null hypothesis is that the probability of success is 0.5 (chance level)
            if len(df_trial_exp1) == 0:
                print("No trials for this combination")
                continue
            p_value = binomtest(df_trial_exp1['picked_target'].sum(), n=len(df_trial_exp1['picked_target']), p=0.5, alternative='two-sided').pvalue

            print("Number of experiment trials:", len(df_trial_exp1))
            print("Success rate: ", len(df_trial_exp1[df_trial_exp1["picked_target"]]) / len(df_trial_exp1))
            print(f'P-value: {p_value}')
            
            print("--------------------")
            print()


In [ ]:
filtered_df = pd.read_csv(f'{dataframe_path}filtered_responses_v{response_version}.csv')
experiment = 1
df_trial_exp = filtered_df[(filtered_df['catch_trial'].isnull()) & (filtered_df['experiment'] == experiment)]
import matplotlib.pyplot as plt

# Calculate the percentage of "picked_target" values by method and mode
percentages = df_trial_exp.groupby(['method', 'mode'])['picked_target'].mean().unstack() * 100

# Plot the bar plot
ax = percentages.plot(kind='bar', color=['orange', 'blue'], width=0.8)
ax.set_ylim(40, 100)  # Set the Y axis to start at 45%
ax.axhline(y=50, color='r', linestyle='--')  # Add a dashed horizontal line at 50%

# Add labels and title
plt.xlabel('Method')
plt.ylabel('Percent Correctly Identified')
plt.title('Experiment 1:\nPercent Correctly Identified by Method')

# Add legend
plt.legend(title='Mode')

# Show the plot
plt.show()

# Iterate over each unique value of the "stimtype" column
for stimtype in df_trial_exp['stimtype'].unique():
    # Filter the dataframe for the current stimtype
    df_stimtype = df_trial_exp[df_trial_exp['stimtype'] == stimtype]
    
    # Calculate the percentage of "picked_target" values by method and mode
    percentages = df_stimtype.groupby(['method', 'mode'])['picked_target'].mean().unstack() * 100
    
    # Plot the bar plot
    ax = percentages.plot(kind='bar', color=['orange', 'blue'], width=0.8)
    ax.set_ylim(40, 100)  # Set the Y axis to start at 45%
    ax.axhline(y=50, color='r', linestyle='--')  # Add a dashed horizontal line at 50%
    
    # Add labels and title
    plt.xlabel('Method')
    plt.ylabel('Percent Correctly Identified')
    plt.title(f'Experiment 1:\nPercent Correctly Identified by Method for {stimtype.capitalize()} Stimuli')
    
    # Add legend
    plt.legend(title='Mode')
    
    # Show the plot
    plt.show()

In [ ]:

import matplotlib.ticker as mtick
human_df = filtered_df[(filtered_df['experiment'] == 1) & (filtered_df['method'] != "tagaki") & (filtered_df['method'] != "mindeye2")]# 
grouped_data = human_df.groupby(['sample', 'subject', 'stimtype', 'mode'])['picked_target'].mean().reset_index()
grouped_data.rename(columns={'picked_target': 'Accuracy'}, inplace=True)
grouped_data['Accuracy'] *= 100

# Function to calculate cumulative distribution
def cumulative_distribution(data):
    # Sort data
    data_sorted = np.sort(data)
    # Calculate cumulative probability for each value in the sorted array
    p = 100. * np.arange(len(data)) / (len(data) - 1)
    return data_sorted, p

# Separate the data into the four groups
vision_complex = grouped_data[(grouped_data['mode'] == 'vision') & (grouped_data['stimtype'] == 'complex')]['Accuracy']
vision_simple = grouped_data[(grouped_data['mode'] == 'vision') & (grouped_data['stimtype'] == 'simple')]['Accuracy']
imagery_complex = grouped_data[(grouped_data['mode'] == 'imagery') & (grouped_data['stimtype'] == 'complex')]['Accuracy']
imagery_simple = grouped_data[(grouped_data['mode'] == 'imagery') & (grouped_data['stimtype'] == 'simple')]['Accuracy']

# Calculate cumulative distributions for each group using the previously defined function
vision_complex_sorted, p_vision_complex = cumulative_distribution(vision_complex)
vision_simple_sorted, p_vision_simple = cumulative_distribution(vision_simple)
imagery_complex_sorted, p_imagery_complex = cumulative_distribution(imagery_complex)
imagery_simple_sorted, p_imagery_simple = cumulative_distribution(imagery_simple)

# Plotting
fig, ax = plt.subplots(figsize=(8, 8), dpi=500)  # Create a square figure
ax.plot(vision_simple_sorted, p_vision_simple, color='green', label='Vision Simple', linewidth=3)
ax.plot(imagery_simple_sorted, p_imagery_simple, color='orange', label='Imagery Simple', linewidth=3)
ax.plot(imagery_complex_sorted, p_imagery_complex, color='red', label='Imagery Complex', linewidth=3)
ax.plot(vision_complex_sorted, p_vision_complex, color='blue', label='Vision Complex', linewidth=3)

for _ in tqdm(range(1000)):
    # Randomize 'picked_target'
    randomized_df = human_df.copy()
    randomized_df['picked_target'] = np.random.rand(len(randomized_df)) < 0.5

    # Group and calculate new accuracies
    randomized_grouped_data = randomized_df.groupby(['sample', 'subject', 'method', 'stimtype', 'mode'])['picked_target'].mean().reset_index()
    randomized_grouped_data.rename(columns={'picked_target': 'Accuracy'}, inplace=True)
    randomized_grouped_data['Accuracy'] *= 100
    # Separate the data into the four groups
    randomized_vision_complex = randomized_grouped_data[(randomized_grouped_data['mode'] == 'vision') & (randomized_grouped_data['stimtype'] == 'complex')]['Accuracy']
    randomized_vision_simple = randomized_grouped_data[(randomized_grouped_data['mode'] == 'vision') & (randomized_grouped_data['stimtype'] == 'simple')]['Accuracy']
    randomized_imagery_complex = randomized_grouped_data[(randomized_grouped_data['mode'] == 'imagery') & (randomized_grouped_data['stimtype'] == 'complex')]['Accuracy']
    randomized_imagery_simple = randomized_grouped_data[(randomized_grouped_data['mode'] == 'imagery') & (randomized_grouped_data['stimtype'] == 'simple')]['Accuracy']

    # Calculate cumulative distributions for each randomized group
    r_vision_complex_sorted, r_p_vision_complex = cumulative_distribution(randomized_vision_complex)
    r_vision_simple_sorted, r_p_vision_simple = cumulative_distribution(randomized_vision_simple)
    r_imagery_complex_sorted, r_p_imagery_complex = cumulative_distribution(randomized_imagery_complex)
    r_imagery_simple_sorted, r_p_imagery_simple = cumulative_distribution(randomized_imagery_simple)

    # Plot the randomized data
    ax.plot(r_vision_simple_sorted, r_p_vision_simple, color="gray", alpha=0.05)
    ax.plot(r_vision_complex_sorted, r_p_vision_complex, color="gray", alpha=0.05)
    ax.plot(r_imagery_simple_sorted, r_p_imagery_simple, color="gray", alpha=0.05)
    ax.plot(r_imagery_complex_sorted, r_p_imagery_complex, color="gray", alpha=0.05)

# Adding labels and title
ax.set_xlabel(f'Percentage of Stimuli\nCorrectly Identified in a 2AFC Task', fontsize=14)
ax.set_ylabel('Cumulative Percentage of\nSamples to Reach Accuracy', fontsize=14)
ax.set_title(f'Cumulative Distribution Plot\nof Human Identification Accuracy (No Takagi or ME2)', fontsize=16)
ax.legend(loc='upper left', fontsize=12)
ax.grid(True)
# plt.tight_layout()
# Format x-axis ticks as percentages
ax.xaxis.set_major_formatter(mtick.PercentFormatter())

# Format y-axis ticks as percentages
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
# Set the x-axis limit to end at 100%
ax.set_xlim(0, 100)

# Set the y-axis limit to end at 100%
ax.set_ylim(0, 100)

# Ensure the aspect ratio of the plot is equal
ax.set_aspect('equal', adjustable='box')

plt.show()


In [ ]:

# Iterate over each experiment
experiment = 1
for mode in ["vision", "imagery"]:
    
    print(f"Mode: {mode}")
    for stimtype in ["simple", "complex", "concept"]:
        print(f"Stimtype: {stimtype}")
        # Filter the data for the current method, mode, and experiment
        df_trial_exp1 = df_trial_exp[(df_trial_exp['method'] != "takagi") & (df_trial_exp['method'] != "mindeye2") & (df_trial_exp['experiment'] == experiment) & (df_trial_exp['mode'] == mode) & (df_trial_exp['stimtype'] == stimtype)]
        
        if len(df_trial_exp1) == 0:
                print("No trials for this combination")
                continue
        # Perform a binomial test
        # The null hypothesis is that the probability of success is 0.5 (chance level)
        p_value = binomtest(df_trial_exp1['picked_target'].sum(), n=len(df_trial_exp1['picked_target']), p=0.5, alternative='two-sided').pvalue

        print("Number of experiment trials:", len(df_trial_exp1))
        print("Success rate: ", len(df_trial_exp1[df_trial_exp1["picked_target"]]) / len(df_trial_exp1))
        print(f'P-value: {p_value}')
        
        print("--------------------")
        print()


In [ ]:
# Load filtered responses
experiment = 2
df_trial_exp = filtered_df[(filtered_df['catch_trial'].isnull()) & (filtered_df['experiment'] == experiment)]
print(len(df_trial_exp))
# # Iterate over each method
# for method in df_trial_exp['method'].unique():
#         print(f"Method: {method}")
print("--------------------")
for stimtype in ["simple", "complex", "concept"]:
        print(f"Stimtype: {stimtype}")
        # Filter the data for the current method, mode, and experiment
        df_trial_exp_stim = df_trial_exp[(df_trial_exp['stimtype'] == stimtype)]
        # Perform a binomial test
        # The null hypothesis is that the probability of success is 0.5 (chance level)
        if len(df_trial_exp_stim) == 0:
                print("No trials for this combination")
                continue
        p_value = binomtest(df_trial_exp_stim['picked_target'].sum(), n=len(df_trial_exp_stim['picked_target']), p=0.5, alternative='two-sided').pvalue

        print("Number of experiment trials:", len(df_trial_exp_stim))
        print("Success rate: ", len(df_trial_exp_stim[df_trial_exp_stim["picked_target"]]) / len(df_trial_exp_stim))
        print(f'P-value: {p_value}')

        print("--------------------")
        print()


In [ ]:
import matplotlib.pyplot as plt
df_trial_exp3 = filtered_df[(filtered_df['catch_trial'].isnull()) & (filtered_df['experiment'] == 3)]
# Calculate the average similarity and confidence for each method and stimtype
averages = []
for method in df_trial_exp3['method'].unique():
    # for stimtype in df_trial_exp3['stimtype'].unique():
    category_df = df_trial_exp3[(df_trial_exp3["method"] == method)]
    avg_vision_similarity = category_df['target_similarity'].mean()
    avg_imagery_similarity = category_df['distractor_similarity'].mean()
    avg_vision_confidence = category_df['target_confidence'].mean()
    avg_imagery_confidence = category_df['distractor_confidence'].mean()
    print(f"Method: {method}, Average Vision Similarity: {avg_vision_similarity:.3f}, Average Vision Confidence: {avg_vision_confidence:.3f}, Average Imagery Similarity: {avg_imagery_similarity:.3f}, Average Imagery Confidence: {avg_imagery_confidence:.3f}")
    averages.append((method, stimtype, avg_vision_similarity, avg_imagery_similarity))

# Create a list of methods and their corresponding average similarity values for vision and imagery
methods = []
vision_similarities = []
imagery_similarities = []

for method, _, avg_vision_similarity, avg_imagery_similarity in averages:
    methods.append(method)
    vision_similarities.append(avg_vision_similarity)
    imagery_similarities.append(avg_imagery_similarity)

# Set the width of the bars
bar_width = 0.35

# Set the positions of the bars on the X-axis
r1 = np.arange(len(methods))
r2 = [x + bar_width for x in r1]

# Plot the bars
plt.bar(r1, vision_similarities, color='blue', width=bar_width, label='Vision')
plt.bar(r2, imagery_similarities, color='orange', width=bar_width, label='Imagery')

# Add labels and title
plt.xlabel('Methods')
plt.ylabel('Average Similarity')
plt.title('Experiment 3:\nAverage Similarity for Vision and Imagery')
plt.xticks([r + bar_width/2 for r in range(len(methods))], methods)

# Add legend
plt.legend()

# Show the plot
plt.show()

# Iterate over each unique value of the "stimtype" column
for stimtype in df_trial_exp3['stimtype'].unique():
    # Filter the dataframe for the current stimtype
    df_stimtype = df_trial_exp3[df_trial_exp3['stimtype'] == stimtype]
    
    # Calculate the average similarity and confidence for each method
    averages = []
    for method in df_stimtype['method'].unique():
        category_df = df_stimtype[df_stimtype["method"] == method]
        avg_vision_similarity = category_df['target_similarity'].mean()
        avg_imagery_similarity = category_df['distractor_similarity'].mean()
        avg_vision_confidence = category_df['target_confidence'].mean()
        avg_imagery_confidence = category_df['distractor_confidence'].mean()
        averages.append((method, stimtype, avg_vision_similarity, avg_imagery_similarity))
    
    # Create a list of methods and their corresponding average similarity values for vision and imagery
    methods = []
    vision_similarities = []
    imagery_similarities = []

    for method, _, avg_vision_similarity, avg_imagery_similarity in averages:
        methods.append(method)
        vision_similarities.append(avg_vision_similarity)
        imagery_similarities.append(avg_imagery_similarity)

    # Set the positions of the bars on the X-axis
    r1 = np.arange(len(methods))
    r2 = [x + bar_width for x in r1]

    # Plot the bars
    plt.bar(r1, vision_similarities, color='blue', width=bar_width, label='Vision')
    plt.bar(r2, imagery_similarities, color='orange', width=bar_width, label='Imagery')

    # Add labels and title
    plt.xlabel('Methods')
    plt.ylabel('Average Similarity')
    plt.title(f'Experiment 3:\nAverage Similarity for Vision and Imagery ({stimtype.capitalize()})')
    plt.xticks([r + bar_width/2 for r in range(len(methods))], methods)

    # Add legend
    plt.legend()

    # Show the plot
    plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np

# human_df = pd.read_csv(f'experiments/dataframes/responses_v4_clean.csv')
human_df = filtered_df[(filtered_df['catch_trial'].isnull()) & (filtered_df['experiment'] == 3)]
# Grouping and averaging
grouped_df = human_df.groupby(['method', 'sample']).mean().reset_index()

# Define a clear mapping for method display names in the legend
method_display_names = {
    "mirage" :  "MIRAGE",
    "mindeye": "MindEye1",
    "braindiffuser": "Brain Diffuser",
    "iCNN" : "iCNN",
    "mindeye2": "MindEye2",
    "takagi" : "Takagi et al."
}

# Prepare the plot
fig, ax = plt.subplots(figsize=(8, 8), dpi=500)  # Create a square figure
bright_palette = sns.color_palette('bright', len(method_display_names))
markers = ['o', 's', 'D', '^', 'p', '*']

# First, plot the scatter and PCA lines for each method
for i, (method_code, method_display) in enumerate(method_display_names.items()):
    method_data = grouped_df[grouped_df['method'] == method_code]
    # Scatter plot for individual points
    sns.scatterplot(data=method_data, x='target_similarity', y='distractor_similarity', color=bright_palette[i], 
                    label=method_display, s=100, marker=markers[i], alpha=0.7, ax=ax)

    # PCA for best fit line
    pca = PCA(n_components=2)
    points = method_data[['target_similarity', 'distractor_similarity']].dropna()
    pca.fit(points)
    center = pca.mean_
    pc1 = pca.components_[0]
    line_x = np.linspace(points['target_similarity'].min() - 0.5, points['target_similarity'].max() + 0.5, 2)
    line_y = (pc1[1] / pc1[0]) * (line_x - center[0]) + center[1]
    ax.plot(line_x, line_y, color=bright_palette[i], linewidth=2)

# Then, plot the large dots for the mean of each method, ensuring they appear on top
for i, (method_code, method_display) in enumerate(method_display_names.items()):
    method_data = grouped_df[grouped_df['method'] == method_code]
    avg_target_similarity = method_data['target_similarity'].mean()
    avg_distractor_similarity = method_data['distractor_similarity'].mean()
    ax.scatter(avg_target_similarity, avg_distractor_similarity, color=bright_palette[i], marker=markers[i], s=350, edgecolors='black', zorder=5)

ax.set_title('Average Human Similarity Scores\nBetween Vision and Imagery', pad=10, fontsize=16)
ax.set_xlabel('Human Similarity Score\n(Vision Trials)', fontsize=14)
ax.set_ylabel('Human Similarity Score\n(Imagery Trials)', fontsize=14)
plt.tight_layout()
ax.grid(True)
ax.set_xlim(0, 0.7)
ax.set_ylim(0, 0.7)
ax.set_aspect('equal', adjustable='box')
ax.plot([0, 0.7], [0, 0.7], 'k--')  # Line at unity

# Adjusting legend to make sure it represents the latest plotted elements correctly
handles, labels = ax.get_legend_handles_labels()
# Ensuring that new handles created for the legend represent the latest plot elements with the correct zorder
new_handles = [plt.Line2D([], [], marker=marker, color='w', markerfacecolor=bright_palette[i], markersize=10, alpha=1) for i, marker in enumerate(markers[:len(method_display_names)])]
ax.legend(new_handles, labels[:len(method_display_names)], bbox_to_anchor=(0, 1), loc='upper left', fontsize=12)

plt.show()


In [ ]:
import matplotlib.pyplot as plt
df_trial_exp3 = filtered_df[(filtered_df['catch_trial'].isnull()) & (filtered_df['experiment'] == 4)]
# Calculate the average similarity and confidence for each method and stimtype
averages = []
for method in df_trial_exp3['stim2_method'].unique():
    # for stimtype in df_trial_exp3['stimtype'].unique():
    category_df = df_trial_exp3[(df_trial_exp3["stim2_method"] == method)]
    avg_target_similarity = category_df['target_similarity'].mean()
    avg_distractor_similarity = category_df['distractor_similarity'].mean()
    avg_target_confidence = category_df['target_confidence'].mean()
    avg_distractor_confidence = category_df['distractor_confidence'].mean()
    print(f"Method: {method}, Average Method Similarity: {avg_target_similarity:.3f}, Average Method Confidence: {avg_target_confidence:.3f}, Average Distractor Similarity: {avg_distractor_similarity:.3f}, Average Distractor Confidence: {avg_distractor_confidence:.3f}")
    averages.append((method, stimtype, avg_target_similarity, avg_distractor_similarity))

# Create a list of methods and their corresponding average similarity values for vision and imagery
methods = []
target_similarities = []
distractor_similarities = []

for method, _, avg_target_similarity, avg_distractor_similarity in averages:
    methods.append(method)
    target_similarities.append(avg_target_similarity)
    distractor_similarities.append(avg_distractor_similarity)

# Set the width of the bars
bar_width = 0.35

# Set the positions of the bars on the X-axis
r1 = np.arange(len(methods))
r2 = [x + bar_width for x in r1]

# Plot the bars
plt.bar(r1, target_similarities, color='blue', width=bar_width, label='Target')
plt.bar(r2, distractor_similarities, color='orange', width=bar_width, label='Distractor')

# Add labels and title
plt.xlabel('Methods')
plt.ylabel('Average Similarity')
plt.title('Experiment 4: Average Similarity Head to Head: Concepts')
plt.xticks([r + bar_width/2 for r in range(len(methods))], methods)

# Add legend
plt.legend()

# Show the plot
plt.show()

